In [21]:
import pandas as pd
import datetime
import calendar
import statsmodels.formula.api as sm

In [2]:
features_df = pd.read_csv("features.csv")
stores_df = pd.read_csv("stores.csv")
train_df = pd.read_csv("train.csv")

In [3]:
seasons_dict = {
    1:"Winter",
    2:"Winter",
    3:"Spring",
    4:"Spring",
    5:"Spring",
    6:"Summer",
    7:"Summer",
    8:"Summer",
    9:"Fall",
    10:"Fall",
    11:"Fall",
    12:"Winter"
}

In [4]:
master_df = train_df.merge(stores_df, on='Store', how='left')
master_df = master_df.merge(features_df, on=['Store', 'Date'], how='left')

In [5]:
master_df['MarkDown1'] = master_df['MarkDown1'].fillna(0)
master_df['MarkDown2'] = master_df['MarkDown2'].fillna(0)
master_df['MarkDown3'] = master_df['MarkDown3'].fillna(0)
master_df['MarkDown4'] = master_df['MarkDown4'].fillna(0)
master_df['MarkDown5'] = master_df['MarkDown5'].fillna(0)
master_df['Date'] = pd.to_datetime(master_df['Date'], format='%Y-%m-%d')
master_df['Week_Number'] = master_df['Date'].dt.week + 1
master_df['Quarter'] = master_df['Date'].dt.quarter
master_df['Month'] = master_df['Date'].dt.month.apply(lambda x: calendar.month_abbr[x])
master_df['Season'] = (master_df['Date'].apply(lambda dt: (dt.month%12 + 3)//3)).map(seasons_dict)

In [7]:
master_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Week_Number,Quarter,Month,Season
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,0.0,...,0.0,0.0,0.0,211.096358,8.106,False,6,1,Feb,Winter
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,0.0,...,0.0,0.0,0.0,211.242170,8.106,True,7,1,Feb,Winter
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,0.0,...,0.0,0.0,0.0,211.289143,8.106,False,8,1,Feb,Winter
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,0.0,...,0.0,0.0,0.0,211.319643,8.106,False,9,1,Feb,Winter
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,0.0,...,0.0,0.0,0.0,211.350143,8.106,False,10,1,Mar,Winter


In [13]:
master_df = master_df.join(pd.get_dummies(master_df['Quarter'], prefix='Quarter'))

In [14]:
master_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Type,Size,Temperature,Fuel_Price,MarkDown1,...,Unemployment,IsHoliday_y,Week_Number,Quarter,Month,Season,Quarter_1,Quarter_2,Quarter_3,Quarter_4
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,0.0,...,8.106,False,6,1,Feb,Winter,1,0,0,0
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,0.0,...,8.106,True,7,1,Feb,Winter,1,0,0,0
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,0.0,...,8.106,False,8,1,Feb,Winter,1,0,0,0
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,0.0,...,8.106,False,9,1,Feb,Winter,1,0,0,0
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,0.0,...,8.106,False,10,1,Mar,Winter,1,0,0,0


In [28]:
result = sm.ols(formula="Weekly_Sales ~ IsHoliday_x + Fuel_Price + Temperature", data=master_df).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:           Weekly_Sales   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     23.07
Date:                Wed, 18 Jul 2018   Prob (F-statistic):           6.30e-15
Time:                        11:57:25   Log-Likelihood:            -4.8268e+06
No. Observations:              421570   AIC:                         9.654e+06
Df Residuals:                  421566   BIC:                         9.654e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.578e+04    